<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 从零实现的字节对编码（Byte Pair Encoding, BPE）分词器 —— 简单版


- 这是一个独立的笔记本，从零实现流行的字节对编码（Byte Pair Encoding, BPE）分词算法，用于教育目的。这种算法被用于 GPT-2 到 GPT-4、Llama 3 等模型中。
- 关于分词的目的的更多细节，请参考 [第 2 章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)；这里的代码是附加内容，用于解释 BPE 算法。
- OpenAI 为训练原始 GPT 模型实现的 BPE 分词器可以在这里找到 [encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)。
- BPE 算法最初发表于 1994 年，论文为 Philip Gage 的 "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"。
- 如今，大多数项目（包括 Llama 3）都使用 OpenAI 的开源 [tiktoken 库](https://github.com/openai/tiktoken)，因为其计算性能优异；它允许加载预训练的 GPT-2 和 GPT-4 分词器（例如，Llama 3 模型也使用 GPT-4 分词器训练）。
- 上述实现与我在本笔记本中的实现的区别在于，我的实现还包含一个用于训练分词器的函数（用于教育目的）。
- 还有一个名为 [minBPE](https://github.com/karpathy/minbpe) 的实现，支持训练，可能性能更好（我这里的实现主要用于教育目的）；与 `minBPE` 相比，我的实现额外支持加载原始 OpenAI 分词器词表并进行合并。


**这是一个非常简单的实现，仅用于教育目的。[bpe-from-scratch.ipynb](bpe-from-scratch.ipynb) 笔记本包含了一个更复杂（但阅读起来更困难）的实现，其行为与 tiktoken 一致。**


&nbsp;
# 1. 字节对编码（BPE）的主要思想


- BPE 的主要思想是将文本转换为整数表示（token ID），用于大型语言模型（LLM）训练（参见 [第 2 章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)）。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/bpe-overview.webp" width="600px">


&nbsp;
## 1.1 位与字节


- 在介绍 BPE 算法之前，先引入字节（byte）的概念。
- 考虑将文本转换为字节数组（毕竟 BPE 的全称是 "byte pair encoding"）： 


In [1]:
text = "This is some text"
byte_ary = bytearray(text, "utf-8")
print(byte_ary)

bytearray(b'This is some text')


- 当我们对 `bytearray` 对象调用 `list()` 时，每个字节会被视为一个独立的元素，结果是一个整数列表，对应于每个字节的值：


In [2]:
ids = list(byte_ary)
print(ids)

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]


- 这是一种将文本转换为 token ID 表示的有效方法，可以用于 LLM 的嵌入层。
- 然而，这种方法的缺点是每个字符都会生成一个 ID（对于短文本来说 ID 数量很多！）。
- 也就是说，对于一个 17 个字符的输入文本，我们需要使用 17 个 token ID 作为 LLM 的输入：


In [3]:
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

Number of characters: 17
Number of token IDs: 17


- 如果你以前使用过 LLM，你可能知道 BPE 分词器有一个词汇表，其中为整个单词或子词分配 token ID，而不是为每个字符分配。
- 例如，GPT-2 分词器将相同的文本（"This is some text"）分词为 4 个 token，而不是 17 个：`1212, 318, 617, 2420`。
- 你可以使用交互式 [tiktoken 应用](https://tiktokenizer.vercel.app/?model=gpt2) 或 [tiktoken 库](https://github.com/openai/tiktoken) 来验证：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/bpe-from-scratch/tiktokenizer.webp" width="600px">

```python
import tiktoken

gpt2_tokenizer = tiktoken.get_encoding("gpt2")
gpt2_tokenizer.encode("This is some text")
# 输出 [1212, 318, 617, 2420]

```

- 由于一个字节由 8 位组成，因此单个字节可以表示 2<sup>8</sup> = 256 个可能的值，范围从 0 到 255。
- 你可以通过执行代码 `bytearray(range(0, 257))` 来验证，会提示 `ValueError: byte must be in range(0, 256)`。
- BPE 分词器通常将这 256 个值作为其前 256 个单字符 token；你可以通过运行以下代码来直观地查看：


```python
import tiktoken
gpt2_tokenizer = tiktoken.get_encoding("gpt2")

for i in range(300):
    decoded = gpt2_tokenizer.decode([i])
    print(f"{i}: {decoded}")
"""
prints:
0: !
1: "
2: #
...
255: �  # <---- single character tokens up to here
256:  t
257:  a
...
298: ent
299:  n
"""
```

- 上述代码中，请注意索引 256 和 257 并不是单字符值，而是双字符值（一个空格 + 一个字母），这是原始 GPT-2 BPE 分词器的一个小缺陷（在 GPT-4 分词器中已得到改进）。


&nbsp;
## 1.2 构建词汇表


- BPE 分词算法的目标是构建一个常见子词的词汇表，例如 `298: ent`（可以在 *entangle, entertain, enter, entrance, entity, ...* 中找到），甚至包括完整单词，如 

```
318: is
617: some
1212: This
2420: text
```

- BPE 算法最早发表于 1994 年，由 Philip Gage 提出，论文为 "[A New Algorithm for Data Compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)"。
- 在进入实际代码实现之前，今天用于 LLM 分词器的 BPE 形式可以总结如下：


&nbsp;
## 1.3 BPE 算法概述

**1. 识别高频对**
- 在每次迭代中，扫描文本以找到最常出现的字节（或字符）对。

**2. 替换并记录**
- 用一个新的占位 ID（尚未使用的 ID，例如，如果从 0...255 开始，第一个占位符为 256）替换该字节对。
- 在查找表中记录这个映射关系。
- 查找表的大小是一个超参数，也称为“词汇表大小”（对于 GPT-2 为 50,257）。

**3. 重复直到无法继续**
- 不断重复步骤 1 和 2，持续合并出现频率最高的字节对。
- 当无法进一步压缩时停止（例如，没有任何字节对出现超过一次）。

**解压（解码）**
- 要恢复原始文本，通过查找表将每个 ID 替换回对应的字节对，逆向操作即可。


&nbsp;
## 1.4 BPE 算法示例

### 1.4.1 编码部分的具体示例（步骤 1 & 2）

- 假设我们有文本（训练数据集）`the cat in the hat`，希望从中为 BPE 分词器构建词汇表。

**迭代 1**

1. 识别高频对
  - 在此文本中，"th" 出现了两次（在开头和第二个 "e" 前）。

2. 替换并记录
  - 用一个尚未使用的新 token ID 替换 "th"，例如 256。
  - 新文本为：`<256>e cat in <256>e hat`
  - 新词汇表为：

```
  0: ...
  ...
  256: "th"
```

**迭代 2**

1. **识别高频对**  
   - 在文本 `<256>e cat in <256>e hat` 中，字节对 `<256>e` 出现了两次。

2. **替换并记录**  
   - 用一个尚未使用的新 token ID 替换 `<256>e`，例如 `257`。  
   - 新文本为：

     ```
     <257> cat in <257> hat
     ```
   - The updated vocabulary is:
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     ```

**迭代 3**

1. **识别高频对**  
   - 在文本 `<257> cat in <257> hat` 中，字节对 `<257> ` 出现了两次（一次在开头，一次在 “hat” 前）。

2. **替换并记录**  
   - 用一个尚未使用的新 token ID 替换 `<257> `，例如 `258`。  

   - 新文本为：

     ```
     <258>cat in <258>hat
     ```
   - The updated vocabulary is:
     ```
     0: ...
     ...
     256: "th"
     257: "<256>e"
     258: "<257> "
     ```
     
- 依此类推

&nbsp;
### 1.4.2 解码部分的具体示例（步骤 3）

- 为了恢复原始文本，我们通过逆向过程，将每个 token ID 替换为其对应的字节对，顺序与引入时相反。
- 从最终压缩文本开始：`<258>cat in <258>hat`
- 替换 `<258>` → `<257> `：`<257> cat in <257> hat`  
- 替换 `<257>` → `<256>e`：`<256>e cat in <256>e hat`
- 替换 `<256>` → "th"：`the cat in the hat`


&nbsp;
## 2. 一个简单的 BPE 实现


- 下面是一个 Python 类实现了上述 BPE 算法，模仿了 `tiktoken` 的 Python 用户界面。
- 注意，上述编码部分描述了通过 `train()` 进行的原始训练步骤；然而，`encode()` 方法的工作方式类似（虽然看起来稍微复杂一些，因为涉及特殊 token 的处理）：

1. 将输入文本拆分为单个字节
2. 当相邻 token（字节对）匹配已学习的 BPE 合并表中的任何一对时，重复查找并替换（合并）（从最高到最低“排名”，即学习顺序）
3. 继续合并，直到无法再应用合并
4. 最终的 token ID 列表即为编码输出


In [4]:
from collections import Counter, deque
from functools import lru_cache


class BPETokenizerSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

    def train(self, text, vocab_size, allowed_special={"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with 'Ġ'
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including 'Ġ' if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]

        # Extend unique_chars with characters from processed_text that are not already included
        unique_chars.extend(char for char in sorted(set(processed_text)) if char not in unique_chars)

        # Optionally, ensure 'Ġ' is included if it is relevant to your text processing
        if 'Ġ' not in unique_chars:
            unique_chars.append('Ġ')

        # Now create the vocab and inverse vocab dictionaries
        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:  # No more pairs to merge. Stopping training.
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def encode(self, text):
        """
        Encode the input text into a list of token IDs.

        Args:
            text (str): The text to encode.

        Returns:
            List[int]: The list of token IDs.
        """
        tokens = []
        # Split text into tokens, keeping newlines intact
        words = text.replace("\n", " \n ").split()  # Ensure '\n' is treated as a separate token

        for i, word in enumerate(words):
            if i > 0 and not word.startswith("\n"):
                tokens.append("Ġ" + word)  # Add 'Ġ' to words that follow a space or newline
            else:
                tokens.append(word)  # Handle first word or standalone '\n'

        token_ids = []
        for token in tokens:
            if token in self.inverse_vocab:
                # token is contained in the vocabulary as is
                token_id = self.inverse_vocab[token]
                token_ids.append(token_id)
            else:
                # Attempt to handle subword tokenization via BPE
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)

        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        can_merge = True
        while can_merge and len(token_ids) > 1:
            can_merge = False
            new_tokens = []
            i = 0
            while i < len(token_ids) - 1:
                pair = (token_ids[i], token_ids[i + 1])
                if pair in self.bpe_merges:
                    merged_token_id = self.bpe_merges[pair]
                    new_tokens.append(merged_token_id)
                    # Uncomment for educational purposes:
                    # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                    i += 2  # Skip the next token as it's merged
                    can_merge = True
                else:
                    new_tokens.append(token_ids[i])
                    i += 1
            if i < len(token_ids):
                new_tokens.append(token_ids[i])
            token_ids = new_tokens

        return token_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for token_id in token_ids:
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token.startswith("Ġ"):
                # Replace 'Ġ' with a space
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced


- 上述 `BPETokenizerSimple` 类中有大量代码，本笔记本不讨论其详细实现，但下一节提供了使用方法的简要概述，以便更好地理解类的方法。


## 3. BPE 实现使用教程


- 实际上，我强烈推荐使用 [tiktoken](https://github.com/openai/tiktoken)，因为我上面的实现主要关注可读性和教育目的，而不是性能。
- 不过，使用方法与 tiktoken 大致相似，只是 tiktoken 没有训练方法。
- 下面通过一些示例来看我上面实现的 `BPETokenizerSimple` Python 代码是如何工作的（本笔记本不讨论代码的详细实现）。


### 3.1 训练、编码与解码


- 首先，我们考虑一些示例文本作为训练数据集：


In [5]:
import os
import urllib.request

if not os.path.exists("../01_main-chapter-code/the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "../01_main-chapter-code/the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

with open("../01_main-chapter-code/the-verdict.txt", "r", encoding="utf-8") as f: # added ../01_main-chapter-code/
    text = f.read()

- 接下来，让我们初始化并训练 BPE 分词器，词汇表大小为 1,000。
- 注意，由于前面讨论的字节值，词汇表默认已经有 255 个条目，因此我们实际上只是在“学习”745 个词汇条目。
- 作为对比，GPT-2 的词汇表有 50,257 个 token，GPT-4 的词汇表有 100,256 个 token（tiktoken 中的 `cl100k_base`），GPT-4o 使用 199,997 个 token（tiktoken 中的 `o200k_base`）；它们的训练集远比我们上面简单示例文本大得多。


In [6]:
tokenizer = BPETokenizerSimple()
tokenizer.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})

- 你可能想查看词汇表内容（但请注意，这会生成一个很长的列表）。


In [7]:
# print(tokenizer.vocab)
print(len(tokenizer.vocab))

1000


- 该词汇表是通过合并 742 次生成的（约等于 `1000 - len(range(0, 256))`）。


In [8]:
print(len(tokenizer.bpe_merges))

742


- 这意味着前 256 个条目是单字符 token。


- 接下来，让我们使用创建好的合并表，通过 `encode` 方法对一些文本进行编码：


In [9]:
input_text = "Jack embraced beauty through art and life."
token_ids = tokenizer.encode(input_text)
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [10]:
print("Number of characters:", len(input_text))
print("Number of token IDs:", len(token_ids))

Number of characters: 42
Number of token IDs: 20


- 从上面的长度可以看出，一个 42 个字符的句子被编码为 20 个 token ID，相比基于字符字节的编码，有效地将输入长度减少了大约一半。


- 注意，词汇表本身在 `decode()` 方法中被使用，使我们能够将 token ID 映射回文本：


In [11]:
print(token_ids)

[424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]


In [12]:
print(tokenizer.decode(token_ids))

Jack embraced beauty through art and life.


- 遍历每个 token ID 可以让我们更好地理解 token ID 是如何通过词汇表解码的：


In [13]:
for token_id in token_ids:
    print(f"{token_id} -> {tokenizer.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .


- 正如我们所看到的，大多数 token ID 代表 2 个字符的子词；这是因为训练数据文本很短，重复词不多，并且我们使用了相对较小的词汇表大小。


- 总结来说，调用 `decode(encode())` 应该能够重现任意输入文本：


In [14]:
tokenizer.decode(tokenizer.encode("This is some text."))

'This is some text.'

&nbsp;
# 4. 结论


- 就这样！这就是 BPE 的基本工作原理，同时包含了用于创建新分词器的训练方法。
- 希望你觉得这个简短教程对教育目的有用；如果有任何问题，请随时在 [这里](https://github.com/rasbt/LLMs-from-scratch/discussions/categories/q-a) 发起新的讨论。

**这是一个非常简单的实现，仅用于教育目的。[bpe-from-scratch.ipynb](bpe-from-scratch.ipynb) 笔记本包含了一个更复杂（但阅读起来更困难）的实现，其行为与 tiktoken 一致。**
